In [2]:
import netCDF4 as nc
import pandas as pd
import numpy as np

# Open the NetCDF file
ds = nc.Dataset('IonianTemperature.nc', 'r')  # Replace 'path_to_your_file.nc' with the actual file path

print(ds)



# Initialize a dictionary to hold data
# List all variables
# print("Variables in the file:")
# for var in ds.variables:
#     print(var, ds.variables[var])






# Variables:

# 1st
# find given density at depth using equation

# Validate This:
# P = P_0 + ρgh // pressure, P_0 in hPa
def findPressure(depth):
    P_0 = 1013.25
    density = 1025 # can slightly vary (kg/m^3)
    g = 9.81 # m/s^2
    h = depth # m
    return P_0 + density*g*h

# def findDensity(lat,long,depth):
#     pressure = findPressure(depth)
#     temp = 
#     salinity = 

# 2nd
# find given velocity at depth using data

# 3rd
# Factor EZ rising speed, then slow, then with provisions
# choice of complexity and accuracy

# 4th
# Model Path using constant (then decaying speeds)
# Path: { Buoyancy, Drag, || Vectors}

# 5th, Simulations
# Error
# Carribbean Case Study?
# Cases
# Error Definition and Generalization
# Finally Visualization



# Close the dataset
ds.close()
print(ds.variables.keys())

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.11
    title: daily mean fields from Global Ocean Physics Analysis and Forecast updated Daily
    institution: Mercator Ocean International
    producer: CMEMS - Global Monitoring and Forecasting Centre
    source: MOI GLO12
    credit: E.U. Copernicus Marine Service Information (CMEMS)
    contact: https://marine.copernicus.eu/contact
    references: http://marine.copernicus.eu
    subset:source: ARCO data downloaded from the Marine Data Store using the MyOcean Data Portal
    subset:productId: GLOBAL_ANALYSISFORECAST_PHY_001_024
    subset:datasetId: cmems_mod_glo_phy-thetao_anfc_0.083deg_P1D-m_202211
    subset:date: 2024-02-04T22:51:31.778Z
    dimensions(sizes): depth(37), latitude(97), longitude(50), time(8)
    variables(dimensions): float32 depth(depth), float32 latitude(latitude), float32 longitude(longitude), float32 thetao(time, depth, latitude, longitude), float64 tim